# MLOps - Cloud Orbit - Crypto forecast

![Orbit](assets/orbit.png)

In [ ]:
#!pip install cryptowatch-sdk
#!pip install google-cloud-aiplatform
!mkdir -p $HOME/.cw
!echo "apikey: <to_do_developer>" > $HOME/.cw/credentials.yml
!cat  $HOME/.cw/credentials.yml

In [ ]:
import cryptowatch as cw
import pandas as pd
from datetime import datetime, timedelta
from google.cloud import bigquery
from google.cloud import aiplatform
from matplotlib import pyplot

In [ ]:
MARKET="<to_do_developer>" 
PROJECT_ID = "<to_do_developer>" 
REGION = "<to_do_developer>" 
DATASET = "<to_do_developer>"
TEMP_BUCKET="<to_do_developer>"

In [ ]:
coinbase = cw.markets.list(MARKET)
for market in coinbase.markets:

    try:
        ticker = "{}:{}".format(market.exchange, market.pair).upper()
        candles = cw.markets.get(ticker, ohlc=True, periods=["1w"])


        close_ts, wkly_open, wkly_close = (
            candles.of_1w[-1][0],
            candles.of_1w[-1][1],
            candles.of_1w[-1][4],
        )

        if wkly_open == 0:
            continue
        perf = (wkly_open - wkly_close) * 100 / wkly_open

    
        if perf >= 10:
            open_ts = datetime.utcfromtimestamp(close_ts) - timedelta(days=7)
            print("{} ganó {:.2f}% desde {}".format(ticker, perf, open_ts))
    except:
        print("Execpcion capturada, pero continuamos ..")

In [ ]:
TICKET="OXTUSD"
TABLE = "candles_15min_"+TICKET
candles = cw.markets.get(MARKET+":"+TICKET, ohlc=True)

In [ ]:
rows_list = []
for x in candles.of_15m:
    close_ts = datetime.utcfromtimestamp(x[0])
    open_value = x[1]
    high_value = x[2]
    low_value = x[3]
    close_value = x[4]
    volume_base = x[5]
    volume_quote = x[6]
    rows_list.append([TICKET,close_ts , open_value , high_value , low_value ,close_value ,volume_base ,volume_quote])
df = pd.DataFrame(rows_list,columns = ["ticket","close_ts" , "open_value" , "high_value" , "low_value" ,"close_value" ,"volume_base" ,"volume_quote" ])

In [ ]:
df

In [ ]:
df['close_value'].plot()
pyplot.show()

In [ ]:
client = bigquery.Client()
table_id = PROJECT_ID+"."+DATASET+"."+ TABLE
job_config = bigquery.LoadJobConfig(    
    schema=[
        bigquery.SchemaField("ticket", bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField("close_ts", bigquery.enums.SqlTypeNames.TIMESTAMP),
        bigquery.SchemaField("open_value", bigquery.enums.SqlTypeNames.FLOAT64),
        bigquery.SchemaField("high_value", bigquery.enums.SqlTypeNames.FLOAT64),
        bigquery.SchemaField("low_value", bigquery.enums.SqlTypeNames.FLOAT64),
        bigquery.SchemaField("close_value", bigquery.enums.SqlTypeNames.FLOAT64),
        bigquery.SchemaField("volume_base", bigquery.enums.SqlTypeNames.FLOAT64),
        bigquery.SchemaField("volume_quote", bigquery.enums.SqlTypeNames.FLOAT64)
    ],
    write_disposition="WRITE_TRUNCATE",
)

job = client.load_table_from_dataframe(
    df, table_id, job_config=job_config
)  
job.result()  

table = client.get_table(table_id)  
print(
    "Operacion OK.  {} filas y {} columnas cargadas en {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

In [ ]:
dataset = aiplatform.TimeSeriesDataset.create(display_name=table_id, bq_source='bq://'+PROJECT_ID+"."+DATASET+"."+ TABLE)
dataset.wait()

In [ ]:
transformations = [
        #{"auto": {"column_name": "ticket"}},
        {"timestamp": {"column_name": "close_ts"}},
        {"numeric": {"column_name": "open_value"}},
        {"numeric": {"column_name": "high_value"}},
        {"numeric": {"column_name": "low_value"}},
        {"numeric": {"column_name": "close_value"}},
        {"numeric": {"column_name": "volume_base"}},
        {"numeric": {"column_name": "volume_quote"}},
    ]
autoMLForecastJob = aiplatform.AutoMLForecastingTrainingJob(display_name='candles_15min_'+TICKET+'AutoMLJob',
                                                            column_transformations=transformations,
                                                            optimization_objective="minimize-rmse")

In [ ]:
model = autoMLForecastJob.run(dataset=dataset,
                      target_column="close_value",
                      time_column="close_ts",
                      time_series_identifier_column="ticket",
                      unavailable_at_forecast_columns=["open_value" , "high_value" , "low_value" ,"close_value" ,"volume_base" ,"volume_quote"],
                      available_at_forecast_columns=["close_ts"],
                      forecast_horizon=10,
                      data_granularity_unit="minute",
                      data_granularity_count=15,
                      time_series_attribute_columns=[],
                      budget_milli_node_hours=1000,
                      model_display_name="candles_15min_"+TICKET+"AutoMLModel")

In [ ]:
#Prepare prediction input table
forecast_horizon=10
data_granularity_count=15
prediction_row_list=[]
for x in range(forecast_horizon):
    close_ts = datetime.now() +  timedelta(minutes = data_granularity_count*forecast_horizon)
    open_value = 0
    high_value = 0
    low_value = 0
    volume_base = 0
    volume_quote = 0
    close_value = None
    prediction_row_list.append([TICKET,close_ts , open_value , high_value , low_value , volume_base ,volume_quote,close_value])
df_pred = pd.DataFrame(prediction_row_list,columns = ["ticket","close_ts" , "open_value" , "high_value" , "low_value" ,"volume_base" ,"volume_quote","close_value"  ])    

In [ ]:
df_pred

In [ ]:
client = bigquery.Client()
table_id = PROJECT_ID+"."+DATASET+"."+ "prediction_input"
job_config = bigquery.LoadJobConfig(    
    schema=[
        bigquery.SchemaField("ticket", bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField("close_ts", bigquery.enums.SqlTypeNames.TIMESTAMP),
        bigquery.SchemaField("open_value", bigquery.enums.SqlTypeNames.FLOAT64),
        bigquery.SchemaField("high_value", bigquery.enums.SqlTypeNames.FLOAT64),
        bigquery.SchemaField("low_value", bigquery.enums.SqlTypeNames.FLOAT64),
        bigquery.SchemaField("volume_base", bigquery.enums.SqlTypeNames.FLOAT64),
        bigquery.SchemaField("volume_quote", bigquery.enums.SqlTypeNames.FLOAT64),
        bigquery.SchemaField("close_value", bigquery.enums.SqlTypeNames.FLOAT64),
    ],
    write_disposition="WRITE_TRUNCATE",
)

job = client.load_table_from_dataframe(
    df_pred, table_id, job_config=job_config
)  
job.result()  

table = client.get_table(table_id)  
print(
    "Operacion OK.  {} filas y {} columnas cargadas en {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

In [ ]:
batchPredJob = aiplatform.BatchPredictionJob.create(job_display_name="automljob",
                                                    model_name="<to_do_developer>",
                                                    instances_format="csv",
                                                    predictions_format="csv",
                                                    bigquery_source="bq://<to_do_developer>"",
                                                    bigquery_destination_prefix="<to_do_developer>"
                                                    )

In [ ]:
%%bigquery pred_df
SELECT
predicted_close_value.value,
close_ts,
FROM `velascoluis-test.prediction_candles_15min_OXTUSDAutoMLModel_2021_06_03T04_15_43_670Z.predictions`

In [ ]:
pred_df

In [ ]:
pred_df['value'].plot()
pyplot.show()